In [1]:
using Pkg; Pkg.activate("/Users/deszoeks/Projects/ASTRAL/lidar/lidar2019")

using Revise
using Dates

# using CSV
# using DataFrames
using DelimitedFiles
using Printf
using PyPlot

  Activating project at `~/Projects/ASTRAL/lidar/lidar2019`


In [7]:
pd = permutedims
m2n(x) = ismissing(x) ? NaN : x

m2n (generic function with 1 method)

In [49]:
# functions for reading hpl files

"count the headlines before a line that starts with ****"
function numheaderlines(file)
    n = 0 
    open(file) do f
        while !startswith( readline(f), "****" )
            n += 1
        end
    end
    return n + 1
end

"""
header = read_streamlinexr_head(file_path)
Read the Halo Photonics StreamLineXR .hpl file header.
"""
function read_streamlinexr_head(file_path)
    # put the header in a Dict
    h = Dict(
        :nlines => countlines(file_path),
        :nheaderlines => 
        # variables defined outside the do block, defaults will be overwritten.
        :ngates => Int32(104),
        :gatelength => 30.0,
        :nrays => Int32(1), # or 40
        :start_time => DateTime(2024,5,2,10,0,0),
        :vel_resolution => 0.0380,
        :headr1 => split("time Azimuth Elevation Pitch Roll"),
        :units1 => split("hours degrees degrees degrees degrees"),
        :headr2 => ["Range Gate", "Doppler", "Intensity", "Beta"],
        :units2 => ["none", "(m/s)", "(SNR + 1)", "(m-1 sr-1)"]
    )

    # open and read the file header, updating the header dict a
    open(file_path) do file
        for _ in 1:2 # skip lines
            readline(file)
        end

        # read number of gates from line 3
        h[:ngates] = parse(Int32, last(split(readline(file)))) # 104
        # read gate length from line 4
        h[:gatelength] = parse(Float64, last(split(readline(file))))
        for _ in 5:6 # skip
            readline(file)
        end

        # read number of rays from line 7
        h[:nrays] = parse(Int32, last(split(readline(file)))) # 1 for Stare or 40 for User
        for _ in 8:9 # skip
            readline(file)
        end

        # read start time from line 10
        tl = split(readline(file))
        h[:start_time] = DateTime(tl[end-1]*tl[end], "yyyymmddHH:MM:SS.ss")
        # read velocity resolution (m/s) line 11
        h[:vel_resolution] = parse(Float64, last(split(readline(file))))
        readline(file) # skip line 12
        # read header 1 from line 13
        hdrln = split(readline(file))[5:end]
        h[:headr1] = hdrln[1:2:end]
        h[:units1] = hdrln[2:2:end]
        readline(file) # skip line 14
        # read header 2 from line 15
        hdrln = split(readline(file))[4:end] # but don't parse it
        h[:headr2] = ["Range Gate", "Doppler", "Intensity", "Beta"]
        h[:units2] = ["none", "(m/s)", "(SNR + 1)", "(m-1 sr-1)"]
        # for _ in 16:17 # skip
        #     readline(file)
        # end
    end

    return h
end

"""
modifying read_streamlinexr_data!(file_path, header, beams)
Read data and fill in the beams.
"""
function read_streamlinexr_stare!(file_path, h, beams, nheaderlines=17)
    # use header information in h
    nlines = h[:nlines]
    ngates = h[:ngates]

    # beams could be rays or times
    nbeams = round(Int, (nlines-nheaderlines) / (1+ngates)) # = nrays*ntimes
    beam_timeangles = zeros(nbeams, 5)
    beam_velrad = zeros(nbeams, ngates, 4)

    # for User wind profiles beam <--> VAD ray
    # for Stare beam <--> time

    # open and read the file
    open(file_path) do file
        for _ in 1:nheaderlines # skip header lines
            readline(file)
        end

        # now read data
        for ibeam = 1:nbeams
            # beam described by a batch of 1+ngates lines
            # Read the beam parameter line
            line = readline(file)
            beam_timeangles[ibeam,:] .= parse.(Float64, split(line))
            # Read each gate in the beam
            for igate = 1:ngates
                line = readline(file)
                beam_velrad[ibeam, igate,:] .= parse.(Float64, split(line))
            end
        end
    end # close the file

    # parse the variables into the dict beams
    # by beam
    beams[:time     ] .= beam_timeangles[:,1] # decimal hours
    beams[:azimuth  ] .= beam_timeangles[:,2] # degrees
    beams[:elevangle] .= beam_timeangles[:,3] # degrees
    beams[:pitch    ] .= beam_timeangles[:,4]
    beams[:roll     ] .= beam_timeangles[:,5]
    # by gate
    beams[:height   ] .= (beam_velrad[1,:,1].+0.5) .* h[:gatelength] # center of gate, assumes same for all beams

    # dependent variables (beam, gate)
    beams[:dopplervel] .= beam_velrad[:,:,2] # m/s
    beams[:intensity ] .= beam_velrad[:,:,3] # SNR + 1
    beams[:beta      ] .= beam_velrad[:,:,4] # m-1 sr-1  backscatter?
end

"""
beams, h = read_streamlinexr(file_path)
Read the Photonics StreamLineXR .hpl file data and header.
"""
function read_streamlinexr_stare(file_path, nheaderlines=17)
    # use header information in h
    h = read_streamlinexr_head(file_path)
    nlines = h[:nlines]
    ngates = h[:ngates]

    # beams could be rays or times
    nbeams = round(Int, (nlines-17) / (1+ngates)) # = nrays*ntimes
    # initialize a beams Dict
    beams = Dict(
        :time      => Vector{Union{Float32,Missing}}(missing, nbeams), # decimal hours
        :azimuth   => Vector{Union{Float32,Missing}}(missing, nbeams), # degrees
        :elevangle => Vector{Union{Float32,Missing}}(missing, nbeams),
        :pitch     => Vector{Union{Float32,Missing}}(missing, nbeams),
        :roll      => Vector{Union{Float32,Missing}}(missing, nbeams),

        :height    => Vector{Union{Float32,Missing}}(missing, ngates), # center of gate

        # dependent variables (beam, gate)
        :dopplervel => Matrix{Union{Float32,Missing}}(missing, nbeams,ngates), # m/s
        :intensity  => Matrix{Union{Float32,Missing}}(missing, nbeams,ngates), # SNR + 1
        :beta       => Matrix{Union{Float32,Missing}}(missing, nbeams,ngates) # m-1 sr-1  backscatter?
        )

    # read file and fill beams with data
    read_streamlinexr_stare!(file_path, h, beams, nheaderlines)
    return beams, h
end

function plot_stare(beams)
    subplot(2,1,1)
    pcolormesh(beams[:time][:], beams[:height][3:end]/1e3, pd(beams[:intensity][:,3:end].-1),
        cmap=ColorMap("RdYlBu_r"))
    ylim([0, 2])
    colorbar()

    X = beams[:dopplervel]
    X[beams[:intensity] .<= 1.02] .= NaN
    subplot(2,1,2)
    pcolormesh(beams[:time][:], beams[:height][4:end]/1e3, pd(X[:,4:end]),
        cmap=ColorMap("RdYlBu_r"), vmin=-3, vmax=3)
    ylim([0, 2])
    colorbar()
    
    tight_layout()
    return gcf()
end


plot_stare (generic function with 1 method)

In [5]:
lidardir = "./data/lidar"
datestamp = "20240508"
datafile = joinpath(lidardir, datestamp, "Stare_116_$(datestamp)_06.hpl")

beams, head = read_streamlinexr_stare(datafile)

(Dict{Symbol, Array{Union{Missing, Float32}}}(:time => [6.0052776f0, 6.0055637f0, 6.0058475f0, 6.0061307f0, 6.006414f0, 6.006697f0, 6.0069833f0, 6.0072665f0, 6.00755f0, 6.0078335f0  …  6.997439f0, 6.997722f0, 6.9980054f0, 6.998289f0, 6.9985723f0, 6.9988556f0, 6.9991417f0, 6.999425f0, 6.999708f0, 6.999992f0], :dopplervel => [0.0382f0 -0.1147f0 … 7.7205f0 -16.7405f0; 0.0f0 -0.1529f0 … -0.8791f0 -17.7343f0; … ; 0.0f0 -0.1529f0 … 0.4969f0 -0.7262f0; 0.0382f0 -0.1529f0 … 9.6698f0 -0.8026f0], :roll => [-1.42f0, -3.15f0, -0.4f0, -1.21f0, -0.4f0, -2.54f0, -0.19f0, -1.83f0, 0.21f0, -3.05f0  …  -1.01f0, -0.3f0, -0.4f0, -1.52f0, -2.44f0, -1.93f0, -0.6f0, 0.32f0, -0.3f0, -0.7f0], :beta => [1.447753f-7 2.634481f-7 … 2.44864f-6 7.376238f-7; 4.074031f-7 5.508617f-7 … 3.648129f-6 1.763072f-6; … ; 1.778243f-7 7.51334f-8 … 3.888082f-6 4.484677f-7; 6.168411f-7 6.081364f-7 … 2.478664f-6 2.857824f-6], :elevangle => [90.02f0, 90.0f0, 90.0f0, 90.0f0, 90.0f0, 90.0f0, 90.0f0, 90.0f0, 90.0f0, 90.0f0  …  90.0f0,

In [51]:
# plot it
for datestamp in filter(startswith("2024"), readdir(lidardir))
    for starefile in filter(startswith("Stare"), readdir(joinpath(lidardir,datestamp)))
        beams, head = read_streamlinexr_stare(starefile)
        clf()
        plot_stare(beams)
        fileout = "Stare_$(datestamp)_06.png"
        savefig(joinpath(lidardir,"plot",fileout))
        print(fileout*", ")
    end
    print("\n")
end

LoadError: IOError: readdir("./data/lidar/.DS_Store"): not a directory (ENOTDIR)